In [1]:
import os
import sys
import glob
from os import write

from tqdm import tqdm
import pandas as pd
import numpy as np
import ast

sys.path.append(os.path.realpath(os.path.abspath('..')))
from iDrink import iDrinkUtilities

In [2]:
drive = iDrinkUtilities.get_drivepath()

root_iDrink = os.path.join(drive, 'iDrink')
root_val = os.path.join(root_iDrink, "validation_root")
root_stat = os.path.join(root_val, '04_Statistics')
root_omc = os.path.join(root_val, '03_data', 'OMC_new', 'S15133')
root_data = os.path.join(root_val, "03_data")
root_logs = os.path.join(root_val, "05_logs")

csv_val_trials = os.path.join(root_logs, 'validation_trials.csv')
df_val_trials = pd.read_csv(csv_val_trials, sep=';')

csv_settings = os.path.join(root_logs, 'validation_settings.csv')
df_settings = pd.read_csv(csv_settings, sep=';')

csv_calib_error = os.path.join(root_logs, 'calib_errors.csv')
df_calib_error = pd.read_csv(csv_calib_error, sep=';')

csv_murphy = os.path.join(root_stat, '02_categorical', 'murphy_measures.csv')
df_murphy = pd.read_csv(csv_murphy, sep=';')

csv_failed_trials = os.path.join(root_stat, '04_failed_trials', 'failed_trials.csv')
df_failed_trials = pd.read_csv(csv_failed_trials, sep=';')

list_identifier = sorted(df_val_trials['identifier'].tolist())

In [3]:
df_failed_trials.columns

Index(['id_s', 'id_p', 'id_t', 'affected', 'side', 'calib_error', 'HPE', 'P2S',
       'OS', 'murphy', 'murphy_omc', 'success', 'fail', 'failed_at', 'reason'],
      dtype='object')

In [10]:
trials_count = df_failed_trials[df_failed_trials['id_s'] =='S001'].groupby(['id_p', 'affected']).size().reset_index(name='count')

In [15]:
df_failed_trials[df_failed_trials['id_s'] == 'S001'].groupby(['failed_at']).size().reset_index(
    name='count')

,failed_at,count
0,HPE,1
1,Murphy,21
2,OS,20


In [9]:
trials_count.style.to_latex()

'\\begin{tabular}{lllr}\n & id_p & affected & count \\\\\n0 & P07 & affected & 41 \\\\\n1 & P07 & unaffected & 42 \\\\\n2 & P08 & affected & 41 \\\\\n3 & P08 & unaffected & 48 \\\\\n4 & P10 & affected & 43 \\\\\n5 & P10 & unaffected & 40 \\\\\n6 & P11 & affected & 41 \\\\\n7 & P11 & unaffected & 41 \\\\\n8 & P12 & affected & 42 \\\\\n9 & P12 & unaffected & 40 \\\\\n10 & P15 & affected & 45 \\\\\n11 & P15 & unaffected & 45 \\\\\n12 & P19 & affected & 49 \\\\\n13 & P19 & unaffected & 45 \\\\\n14 & P241 & affected & 21 \\\\\n15 & P241 & unaffected & 22 \\\\\n16 & P242 & affected & 21 \\\\\n17 & P242 & unaffected & 20 \\\\\n18 & P251 & affected & 20 \\\\\n19 & P251 & unaffected & 21 \\\\\n20 & P252 & affected & 21 \\\\\n21 & P252 & unaffected & 21 \\\\\n\\end{tabular}\n'

In [21]:
df_failed_trials.groupby(['failed_at', 'id_s']).size().reset_index(name='count')

,failed_at,id_s,count
0,HPE,S001,1
1,HPE,S002,3
2,HPE,S004,7
3,HPE,S005,1
4,HPE,S006,3
...,...,...,...
66,P2S,S010,72
67,P2S,S013,354
68,P2S,S014,347
69,P2S,S015,52


509      7.267665
510      7.267665
511      7.267665
512      7.267665
513      7.267665
           ...   
13315   -2.000000
13316   -2.000000
13317   -2.000000
13318   -2.000000
13319   -2.000000
Name: calib_error, Length: 2444, dtype: float64

In [31]:
df_failed_trials.groupby(['id_s', 'id_p'])['calib_error']

In [32]:
df_failed_trials.groupby(['id_s', 'id_p'])['calib_error'].sum().reset_index()

,id_s,id_p,calib_error
0,S001,P07,115.013085
1,S001,P08,100.751708
2,S001,P10,465.115303
3,S001,P11,1638.498202
4,S001,P12,389.787166
...,...,...,...
177,S024,P19,-188.000000
178,S025,P15,-180.000000
179,S025,P19,-188.000000
180,S026,P15,-180.000000


In [18]:
df_failed_trials['success'].sum()

9081

In [19]:
df_failed_trials['fail'].sum()

4239

In [29]:
df_temp = df_failed_trials[['id_p', 'id_s', 'HPE']]
df_temp.loc['HPE'] = df_temp['HPE'].apply(lambda x: 0 if x == 1 else 1 if x == 0 else x)

C:\Users\johan\AppData\Local\Temp\ipykernel_19608\2525283356.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp.loc['HPE'] = df_temp['HPE'].apply(lambda x: 0 if x == 1 else 1 if x == 0 else x)


In [51]:
df_temp = df_failed_trials[['id_s', 'id_p', 'calib_error']].drop_duplicates()
df_temp = df_temp [df_temp['calib_error'] != -2]

In [52]:
df_temp.groupby(['id_p'])['calib_error'].mean().reset_index()

,id_p,calib_error
0,P07,1.095553
1,P08,0.728921
2,P10,2.015446
3,P11,18.226272
4,P12,2.104800
5,P15,1.443933
6,P19,7.354264
7,P241,1.216845
8,P242,1.169626
9,P251,0.354706


In [58]:
df_temp.groupby(['id_p'])['calib_error'].mean().reset_index().style.to_latex()

'\\begin{tabular}{llr}\n & id_p & calib_error \\\\\n0 & P07 & 1.095553 \\\\\n1 & P08 & 0.728921 \\\\\n2 & P10 & 2.015446 \\\\\n3 & P11 & 18.226272 \\\\\n4 & P12 & 2.104800 \\\\\n5 & P15 & 1.443933 \\\\\n6 & P19 & 7.354264 \\\\\n7 & P241 & 1.216845 \\\\\n8 & P242 & 1.169626 \\\\\n9 & P251 & 0.354706 \\\\\n10 & P252 & 1.280272 \\\\\n\\end{tabular}\n'

In [46]:
df_temp[df_temp['id_p'] == 'P19']['calib_error'].mean()

7.354263528827877

In [55]:
df_temp[df_temp['id_p'] == 'P19']['calib_error'].max()

8.021960271265396

In [54]:
df_temp[df_temp['id_p'] == 'P19']['calib_error']

509     7.267665
1279    7.267665
1630    7.283054
1814    7.283054
2417    7.696826
3187    7.696826
3957    7.907884
4727    7.907884
5078    7.697961
5262    7.697961
5446    8.021960
5630    8.021960
6233    6.712056
7003    6.712056
7437    6.246702
7704    6.246702
Name: calib_error, dtype: float64

In [40]:
df_temp.style.to_latex()

'\\begin{tabular}{lllr}\n & id_s & id_p & calib_error \\\\\n0 & S001 & P07 & 1.385700 \\\\\n83 & S001 & P08 & 1.132042 \\\\\n172 & S001 & P10 & 5.603799 \\\\\n255 & S001 & P11 & 19.981685 \\\\\n337 & S001 & P12 & 4.753502 \\\\\n419 & S001 & P15 & 1.730408 \\\\\n509 & S001 & P19 & 7.267665 \\\\\n603 & S001 & P241 & 2.545337 \\\\\n646 & S001 & P242 & 2.525373 \\\\\n687 & S001 & P251 & 0.469230 \\\\\n728 & S001 & P252 & 3.183073 \\\\\n770 & S002 & P07 & 1.385700 \\\\\n853 & S002 & P08 & 1.132042 \\\\\n942 & S002 & P10 & 5.603799 \\\\\n1025 & S002 & P11 & 19.981685 \\\\\n1107 & S002 & P12 & 4.753502 \\\\\n1189 & S002 & P15 & 1.730408 \\\\\n1279 & S002 & P19 & 7.267665 \\\\\n1373 & S002 & P241 & 2.545337 \\\\\n1416 & S002 & P242 & 2.525373 \\\\\n1457 & S002 & P251 & 0.469230 \\\\\n1498 & S002 & P252 & 3.183073 \\\\\n1540 & S003 & P15 & 1.736733 \\\\\n1630 & S003 & P19 & 7.283054 \\\\\n1724 & S004 & P15 & 1.736733 \\\\\n1814 & S004 & P19 & 7.283054 \\\\\n1908 & S005 & P07 & 1.462317 \\\\\n19

In [39]:
df_calib_error

,date,time,id_s,id_p,cam_used,error
0,08.11.2024,11:55:56,S001,P15,"cam1, cam2, cam3, cam4, cam5",1.730408
1,08.11.2024,12:15:35,S003,P15,"cam6, cam7, cam8, cam9, cam10",1.736733
2,08.11.2024,12:26:16,S005,P15,"cam1, cam3, cam5",1.651214
3,08.11.2024,12:36:50,S007,P15,"cam2, cam3, cam4",1.648930
4,08.11.2024,12:47:11,S009,P15,"cam6, cam8, cam10",1.647874
5,08.11.2024,12:57:30,S011,P15,"cam7, cam8, cam9",1.645155
6,08.11.2024,13:03:08,S013,P15,"cam2, cam4",0.754633
7,08.11.2024,13:08:51,S015,P15,"cam7, cam9",0.736514
8,08.11.2024,14:18:59,S001,P19,"cam1, cam2, cam3, cam4, cam5",7.267665
9,08.11.2024,14:41:44,S003,P19,"cam6, cam7, cam8, cam9, cam10",7.283054


In [33]:
df_failed_trials[df_failed_trials['id_p'] in ['P15', 'P19']].groupby(['id_s', 'id_p'])['calib_error'].sum().reset_index()

,id_s,id_p,calib_error
0,S001,P07,115.013085
1,S001,P08,100.751708
2,S001,P10,465.115303
3,S001,P11,1638.498202
4,S001,P12,389.787166
...,...,...,...
177,S024,P19,-188.000000
178,S025,P15,-180.000000
179,S025,P19,-188.000000
180,S026,P15,-180.000000


In [26]:
df_failed_trials['HPE']

0        1
1        1
2        1
3        1
4        1
        ..
13315    1
13316    1
13317    1
13318    1
13319    1
Name: HPE, Length: 13320, dtype: int64

In [25]:
df_temp['HPE']

0        0
1        0
2        0
3        0
4        0
        ..
13315    0
13316    0
13317    0
13318    0
13319    0
Name: HPE, Length: 13320, dtype: int64

In [ ]:
\begin{tabular}{lllr}
id\_s & id\_p & calib\_error \\
S001, S002 & P07 & 1.385700 \\
S001, S002 & P08 & 1.132042 \\
S001, S002 & P10 & 5.603799 \\
S001, S002 & P11 & 19.981685 \\
S001, S002 & P12 & 4.753502 \\
S001, S002 & P15 & 1.730408 \\
S001, S002 & P19 & 7.267665 \\
S001, S002 & P241 & 2.545337 \\
S001, S002 & P242 & 2.525373 \\
S001, S002 & P251 & 0.469230 \\
S001, S002 & P252 & 3.183073 \\
S003, S004 & P15 & 1.736733 \\
S003, S004 & P19 & 7.283054 \\
S005, S006 & P07 & 1.462317 \\
S005, S006 & P08 & 0.758549 \\
S005, S006 & P10 & 0.969596 \\
S005, S006 & P11 & 19.971588 \\
S005, S006 & P12 & 1.415610 \\
S005, S006 & P15 & 1.651214 \\
S005, S006 & P19 & 7.696826 \\
S005, S006 & P241 & 0.885035 \\
S005, S006 & P242 & 0.760580 \\
S005, S006 & P251 & 0.370609 \\
S005, S006 & P252 & 0.758476 \\
S007, S008 & P07 & 1.452339 \\
S007, S008 & P08 & 0.737334 \\
S007, S008 & P10 & 0.959856 \\
S007, S008 & P11 & 20.848321 \\
S007, S008 & P12 & 1.405195 \\
S007, S008 & P15 & 1.648930 \\
S007, S008 & P19 & 7.907884 \\
S007, S008 & P241 & 0.861323 \\
S007, S008 & P242 & 0.792664 \\
S007, S008 & P251 & 0.384466 \\
S007, S008 & P252 & 0.747517 \\
S009, S010 & P15 & 1.647874 \\
S009, S010 & P19 & 7.697961 \\
S011, S012 & P15 & 1.645155 \\
S011, S012 & P19 & 8.021960 \\
S013, S014 & P07 & 0.587715 \\
S013, S014 & P08 & 0.287760 \\
S013, S014 & P10 & 0.528535 \\
S013, S014 & P11 & 12.103493 \\
S013, S014 & P12 & 0.844891 \\
S013, S014 & P15 & 0.754633 \\
S013, S014 & P19 & 6.712056 \\
S013, S014 & P241 & 0.575685 \\
S013, S014 & P242 & 0.599889 \\
S013, S014 & P251 & 0.194517 \\
S013, S014 & P252 & 0.432021 \\
S015, S016 & P07 & 0.589697 \\
S015, S016 & P15 & 0.736514 \\
S015, S016 & P19 & 6.246702 \\
\end{tabular}

In [ ]:
\begin{tabular}{lllr}
id\_s & id\_p & calib\_error & id\_s & id\_p & calib\_error \\
S001, S002 & P07 & 1.385700 & S009, S010 & P15 & 1.647874 \\
S001, S002 & P08 & 1.132042 & S009, S010 & P19 & 7.697961 \\
S001, S002 & P10 & 5.603799 & S011, S012 & P15 & 1.645155 \\
S001, S002 & P11 & 19.981685 & S011, S012 & P19 & 8.021960 \\
S001, S002 & P12 & 4.753502 & S013, S014 & P07 & 0.587715 \\
S001, S002 & P15 & 1.730408 & S013, S014 & P08 & 0.287760 \\
S001, S002 & P19 & 7.267665 & S013, S014 & P10 & 0.528535 \\
S001, S002 & P241 & 2.545337 & S013, S014 & P11 & 12.103493 \\
S001, S002 & P242 & 2.525373 & S013, S014 & P12 & 0.844891 \\
S001, S002 & P251 & 0.469230 & S013, S014 & P15 & 0.754633 \\
S001, S002 & P252 & 3.183073 & S013, S014 & P19 & 6.712056 \\
S003, S004 & P15 & 1.736733 & S013, S014 & P241 & 0.575685 \\
S003, S004 & P19 & 7.283054 & S013, S014 & P242 & 0.599889 \\
S005, S006 & P07 & 1.462317 & S013, S014 & P251 & 0.194517 \\
S005, S006 & P08 & 0.758549 & S013, S014 & P252 & 0.432021 \\
S005, S006 & P10 & 0.969596 & S015, S016 & P07 & 0.589697 \\
S005, S006 & P11 & 19.971588 & S015, S016 & P15 & 0.736514 \\
S005, S006 & P12 & 1.415610 & S015, S016 & P19 & 6.246702 \\
S005, S006 & P15 & 1.651214 & & & \\
S005, S006 & P19 & 7.696826 & & & \\
S005, S006 & P241 & 0.885035 & & & \\
S005, S006 & P242 & 0.760580 & & & \\
S005, S006 & P251 & 0.370609 & & & \\
S005, S006 & P252 & 0.758476 & & & \\
S007, S008 & P07 & 1.452339 & & & \\
S007, S008 & P08 & 0.737334 & & & \\
S007, S008 & P10 & 0.959856 & & & \\
S007, S008 & P11 & 20.848321 & & & \\
S007, S008 & P12 & 1.405195 & & & \\
S007, S008 & P15 & 1.648930 & & & \\
S007, S008 & P19 & 7.907884 & & & \\
S007, S008 & P241 & 0.861323 & & & \\
S007, S008 & P242 & 0.792664 & & & \\
S007, S008 & P251 & 0.384466 & & & \\
S007, S008 & P252 & 0.747517 & & & \\
\end{tabular}